In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/MigrationResults'

In [3]:
# Add analysis date here to apply to dataframe
analysis_date = '20181103'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], 
        df['Somites'], df['ImageMag']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df = full_df.drop(['Label', 'Mean', 'IntDen', 'RawIntDen', 'delete1', 'delete2', 'delete3'], axis=1)
full_df.head()

,,Area,Image,ROI,ExptDate,Treatment,Stains,Embryo,Somites,ImageMag
0,1,71843.804,20180418_SMPD3gRNA1DNA_Pax7NcadLaminin_Emb1_8s...,CntlArea,20180418,SMPD3gRNA1DNA,Pax7NcadLaminin,Emb1,8ss,10x
1,2,51341.649,20180418_SMPD3gRNA1DNA_Pax7NcadLaminin_Emb1_8s...,ExptArea,20180418,SMPD3gRNA1DNA,Pax7NcadLaminin,Emb1,8ss,10x
0,1,80357.839,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb5_9ss_10x,CntlArea,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb5,9ss,10x
1,2,88925.670,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb5_9ss_10x,ExptArea,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb5,9ss,10x
0,1,88178.692,"20180630_SMPD3MO0,8mM_Pax7_Emb4_8ss_10x",CntlArea,20180630,"SMPD3MO0,8mM",Pax7,Emb4,8ss,10x


In [7]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Group dataframe by treatment
df_byTreatment = full_df.groupby(['Treatment', 'Image', 'ROI'])['Image', 'ROI', 'Area', 'Treatment'].mean()

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(df_byTreatment.xs(treatment))

    # Extract Cntl and Expt areas
    area_cntl = df_treatment.xs('CntlArea', level='ROI')
    area_expt = df_treatment.xs('ExptArea', level='ROI')

    # Generate Expt/Cntl Area Ratios
    area_ratios = pd.DataFrame(area_expt / area_cntl)
    area_ratios.columns = ['Expt/Cntl Area']

    # Normalize all migration area values to mean of control group
    norm_cntl = area_cntl/(float(area_cntl.mean()))
    norm_cntl.columns = ['Norm Cntl Area']
    norm_expt = area_expt/(float(area_cntl.mean()))
    norm_expt.columns = ['Norm Expt Area']
    norm_areas = pd.concat([norm_cntl, norm_expt], axis=1, sort=False)

    # Combine processed values into single dataframe and output as csv file analysis date + 'MigrationResults.csv'
    area_cntl = pd.DataFrame(area_cntl)
    area_cntl.columns = ['Cntl Area']
    area_expt = pd.DataFrame(area_expt)
    area_expt.columns = ['Expt Area']
    results = pd.concat([area_cntl, area_expt, area_ratios, norm_cntl, norm_expt], axis=1, sort=True)
    results.to_csv(analysis_date + '_' + treatment + '_MigrationResults.csv')